In [66]:
import theano
import theano.tensor as T
import theano.tensor.nnet as nnet
import numpy as np
import time

import cv2 
import numpy as np
import math
from matplotlib import pyplot as plt
%matplotlib inline
from scipy import signal
import time
from math import sqrt
from skimage import transform
import skimage 

In [67]:
circle = lambda r: np.array([[1 if sqrt((i-r)**2 + (j-r)**2) <= r else 0 for j in range(0, 2*r+1)] for i in range(0, 2*r+1)])

def createM1TrainingFile(M1filter, img, saveFileAs):
    assert(M1filter.shape == img.shape)
    radius = 3
    kernel = circle(radius)
    width, height = img.shape

    theFile = open(saveFileAs, 'w')
    for x in range(radius, width-radius):
        for y in range(radius, height-radius):
            crop = img[x - radius:x + radius + 1, y - radius:y + radius + 1]
            croped_crop = crop[kernel == 1]
            output = str(M1filter[x][y]) + " "
            for pixel in croped_crop:
                output += str(pixel) + " "            
            print>> theFile, output
            
    theFile.close() 

def scaleTo1(img):
    return (img - np.amin(img) ) / float((np.amax(img) - np.amin(img)))

def ZMUV(image):
    """global zero mean unit variance (ZMUV) normalization"""
    image = image - np.mean(image)
    return image/np.std(image)

def plot(img):
    plt.figure(num=None, figsize=(4, 4), dpi=200, facecolor='w', edgecolor='k')
    plt.imshow(img, 'gray')

def rotate90(img):
    cols, rows = img.shape
    xshift = cols/2 - rows/2
    yshift = rows/2 - cols/2
#     rows, cols = img.shape
    M = cv2.getRotationMatrix2D((cols/2,rows/2),90,1)
    M[0][2] = M[0][2] - xshift
    M[1][2] = M[1][2] + yshift    
    dst = cv2.warpAffine(img,M,(cols,rows))

    return dst

def rotate45(img):
    rows, cols = img.shape
    M = cv2.getRotationMatrix2D((cols/2,rows/2),45,1)
    dst = cv2.warpAffine(img,M,(cols, rows))
    if rows > cols:
        bh = int(cols / (2 * np.cos(np.pi/4)))-1
    else:
        bh = int(rows / (2 * np.cos(np.pi/4)))-1
    
    # crop image by new base-height
    
    return dst, bh


In [70]:
def createM1training(imgName, filterName):
    
    img = cv2.imread("./training/preprocess/"+imgName+"_input.jpg", 0)
    M1 = cv2.imread("./training/preprocess/"+imgName+"_"+filterName+".jpg", 0)
    
    # img = ZMUV(img)
    # M1 = ZMUV(M1)

    img = scaleTo1(img)
    M1 = scaleTo1(M1)
    
    # save img; rotate img 90 degrees three times
    createM1TrainingFile(M1, img, "./training/train/"+imgName+"_train0")
    
    img90 = rotate90(img)
    M190 = rotate90(M1)
    createM1TrainingFile(M190, img90, "./training/train/"+imgName+"_train1")
    
    img90 = rotate90(img90)
    M190 = rotate90(M190)
    createM1TrainingFile(M190, img90, "./training/train/"+imgName+"_train2")
    
    img90 = rotate90(img90)
    M190 = rotate90(M190)
    createM1TrainingFile(M190, img90, "./training/train/"+imgName+"_train3")    
    
    # rotate img 45 degrees; crop; save; rotate 90 degrees three times
    img45, bh = rotate45(img)
    M145, bh = rotate45(M1)
    rows, cols = img45.shape
    xcenter = rows/2
    ycenter = cols/2
    cropimg45 = img45[int(xcenter - bh/2):int(xcenter + bh/2), int(ycenter - bh/2):int(ycenter + bh/2)]
    cropM145 = M145[int(xcenter - bh/2):int(xcenter + bh/2), int(ycenter - bh/2):int(ycenter + bh/2)]
    createM1TrainingFile(cropM145, cropimg45, "./training/train/"+imgName+"_train4")
    
    cropimg45 = rotate90(cropimg45)
    cropM145 = rotate90(cropM145)
    createM1TrainingFile(cropM145, cropimg45, "./training/train/"+imgName+"_train5")    

    cropimg45 = rotate90(cropimg45)
    cropM145 = rotate90(cropM145)
    createM1TrainingFile(cropM145, cropimg45, "./training/train/"+imgName+"_train6")    
    
    cropimg45 = rotate90(cropimg45)
    cropM145 = rotate90(cropM145)
    createM1TrainingFile(cropM145, cropimg45, "./training/train/"+imgName+"_train7")        
    


In [72]:
createM1training('img0', "M1")